<a id="item31"></a>

# 204 - Build a Regression Model in Keras

## Load Libs

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
import keras
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.


## Load dataset

In [4]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [5]:
concrete_data.shape

(1030, 9)

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### Split data into predictors and target

In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>

<a id='item32'></a>

## Train Test Split 

In [13]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (721, 8) (721,)
Test set: (309, 8) (309,)


<a id='item33'></a>

## Build a Neural Network

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<a id="item4"></a>

<a id='item34'></a>

## Train and Test the Network

Let's call the function now to create our model.

In [15]:
# build the model
model = regression_model()

In [16]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 479us/step - loss: 1607.5560
Epoch 2/50
721/721 [==============================] - 0s 75us/step - loss: 1585.8975
Epoch 3/50
721/721 [==============================] - 0s 93us/step - loss: 1563.0927
Epoch 4/50
721/721 [==============================] - 0s 107us/step - loss: 1529.8071
Epoch 5/50
721/721 [==============================] - 0s 83us/step - loss: 1477.6572
Epoch 6/50
721/721 [==============================] - 0s 67us/step - loss: 1400.9985
Epoch 7/50
721/721 [==============================] - 0s 61us/step - loss: 1292.6270
Epoch 8/50
721/721 [==============================] - 0s 65us/step - loss: 1152.1209
Epoch 9/50
721/721 [==============================] - 0s 66us/step - loss: 981.3898
Epoch 10/50
721/721 [==============================] - 0s 120us/step - loss: 795.7764
Epoch 11/50
721/721 [==============================] - 0s 145us/step - loss: 618.8229
Epoch 12/50
721/721 [==============================] - 0s 126u

In [17]:
y_predicted = model.predict(X_test)

In [18]:
mean_squared_error(y_test, y_predicted)

140.77657139652393

In [19]:
%%time

# Applying 50 times to calculate mean and std

mean_squared_errors = []
for _ in range(50):
    model = regression_model()
    model.fit(X_train, y_train, epochs=100, verbose=0)
    y_predicted = model.predict(X_test)
    mse = mean_squared_error(y_test, y_predicted)
    mean_squared_errors.append(mse)

CPU times: user 4min 29s, sys: 34.7 s, total: 5min 3s
Wall time: 3min 29s


In [20]:
print('mean and the standard deviation of the mean squared errors - mean: {} and std: {}'.format(np.mean(mean_squared_errors), np.std(mean_squared_errors)))

mean and the standard deviation of the mean squared errors - mean: 85.17850243477746 and std: 21.354494501872832


In [21]:
print('How does the mean of the mean squared errors compare to that from Step A?')
print('before - mean: 163.2768663797643 and std: 54.13841022487779')
print('after - mean: {} and std: {}'.format(np.mean(mean_squared_errors), np.std(mean_squared_errors)))

How does the mean of the mean squared errors compare to that from Step A?
before - mean: 163.2768663797643 and std: 54.13841022487779
after - mean: 85.17850243477746 and std: 21.354494501872832
